# Test create organization

In [ ]:
import sqlmodel as sqm

# Test subjects
from laminhub_rest.core.account import create_organization_account

# Test assets
from laminhub_rest.schema import Account, OrganizationUser

# Test utils
from laminhub_rest.utils._test import create_test_auth, create_test_account
from laminhub_rest.connector import get_engine
from laminhub_rest.main import client
from laminhub_rest.core.account import delete_account
from laminhub_rest._clean_ci import clean_ci
from laminhub_rest.utils._id import base62

## Parametrize

In [ ]:
auth_1 = create_test_auth()
access_token_1 = auth_1["access_token"]
account_1 = create_test_account(handle=auth_1["handle"], access_token=access_token_1)

## Create an organization

### Call create_organization_account

In [ ]:
org_handle_1 = f"lamin.ci.org.{base62(8)}"
create_organization_account(handle=org_handle_1, _access_token=access_token_1)

In [ ]:
organization = None
stmt = sqm.select(Account).where(Account.handle == org_handle_1)
engine = get_engine()
with sqm.Session(engine) as session:
    organization = session.exec(stmt).one()
    assert organization.user_id is None

In [ ]:
stmt = sqm.select(OrganizationUser).where(
    OrganizationUser.organization_id == organization.id
)
organization_user = None
with sqm.Session(engine) as session:
    organization_user = session.exec(stmt).one()
    assert str(organization_user.user_id) == account_1["id"]
    assert organization_user.role == "owner"

### Post method on `/account` route

In [ ]:
org_handle_2 = f"lamin.ci.org.{base62(8)}"
response = client.post(
    f"/account/?handle={org_handle_2}&organization=true",
    headers={"authentication": f"Bearer {access_token_1}"},
).json()

assert response == "success"

## Clean up test assets

In [ ]:
delete_account(account_1["handle"], access_token_1)
clean_ci()